In [1]:
import keras_ocr
import pandas as pd
import cv2 as cv
import pickle
import gc

from os import path
from glob import glob
from math import ceil
from tqdm import tqdm 

from numba import cuda
from multiprocessing import Process


data_dir = path.join('..', 'data')
dataset_dir = path.join(data_dir, 'datasets')
images_dir = path.join(data_dir, 'img')
temp_dir = path.join(data_dir, 'temp')

file_list = glob(path.join(dataset_dir, '*.csv'))
dataset_file = max(file_list, key=path.getctime)
print(dataset_file)

data = data = pd.read_csv(path.join(data_dir, dataset_file), sep='`')

..\data\datasets\data_0.99.csv


In [2]:
# data.info()

In [3]:
data.image_text = [''] * data.shape[0]

In [4]:
list_of_image_files = glob(path.join(images_dir, '*'))

In [5]:
def process_images(paths: list, batch_size=128: int):
    
    pipeline = keras_ocr.pipeline.Pipeline()
    
    results = []
    for i in tqdm(range(ceil(len(paths) / batch_size))):
        gc.collect()
        paths_batch = paths[i*batch_size:(i+1) * batch_size]
        
        filtrated_paths = [path_ for path_ in paths_batch if path_.split('.')[-1] in ['jpg', 'png']]
        posts_id = [path.basename(path_.split('.')[-2]) for path_ in filtrated_paths]
        
        images = [cv.imread(image) for image in filtrated_paths]
        
        ocr_results = pipeline.recognize(images)
        texts = [[tuple_[0] for tuple_ in result if len(tuple_[0]) > 1] for result in ocr_results]
        
        result = list(zip(posts_id, texts))
        for item in result:
            results.append(item)

        with open(path.join(temp_dir, str(i) + '_ocr.pkl'), 'wb') as f:
            pickle.dump(results, f)
            print('Autosaved')
            
    return results

In [8]:
texts = process_images(list_of_image_files[:5], 5)

Looking for C:\Users\techw\.keras-ocr\craft_mlt_25k.h5
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Looking for C:\Users\techw\.keras-ocr\crnn_kurapan.h5


  0%|                                                                                            | 0/1 [00:11<?, ?it/s]


ResourceExhaustedError:  OOM when allocating tensor with shape[4,64,2048,2048] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model_1/basenet.slice1.0/Conv2D (defined at c:\users\techw\appdata\local\programs\python\python38\lib\site-packages\keras_ocr\detection.py:682) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_predict_function_5421]

Function call stack:
predict_function
